# Задание №3

Обучить СНС с помощью Transfer Learning на датасете Food-101.
Использовать тонкую настройку существующей предобученной модели и методы аугментации данных.
Библиотеки: [Python, Tensorflow]

> Возьмем предобученную сеть MobileNetV2 и дообучим в режимах Transfer learning и Fine-tuning
> Будем использовать веса MobileNetV2 на наборе Imagenet
> Целевым набором будет Food01

MobileNet-v2 является сверточной нейронной сетью, глубина которой составляет 53 слоя. Можно загрузить предварительно обученную версию сети, обученной больше чем на миллионе изображений от базы данных [ImageNet](https://image-net.org/). Предварительно обученная сеть может классифицировать изображения в 1 000 категорий объектов. В результате сеть изучила богатые представления функции для широкого спектра изображений. Сеть имеет входной размер изображений 224 224. [Статья](https://habr.com/ru/post/352804/)